### P-2.37

Write a simulator, as in the previous project, but add a Boolean gender ﬁeld and a ﬂoating-point strength ﬁeld to each animal, using an Animal class as a base class.

If two animals of the same type try to collide, then they only create a new instance of that type of animal if they are of different genders. Otherwise, if two animals of the same type and gender try to collide, then only the one of larger strength survives.

In [ ]:
# IMPORTS

import random
import functools
import numpy as np

# LOGGER

import sys
from loguru import logger

logger.remove() # clear the decks
logger.add("debug.log", mode="w", backtrace=False) # specify mode="w" to overwrite each time, or a to append

logger.info("NEW RUN STARTED")

# CLASSES

class Strengths:
    """Generates a set of strengths for creatures in the river system
    Allows each strength value to be retrieved and hence assigned to each creature during
    initialisation of the system"""
    
    def __init__(self, size):
        
        """Generates a sampling distribution based on the Normal distribution"""
        count = 0
        strengths = [-1,2]
        while min(strengths) <= 0 or max(strengths) >= 1: # Ensures we only generate a distribution with values within acceptable range
            count += 1
            if count > 10: raise Exception("Too many attempts to build a normal distribution")
            strengths = np.random.normal(loc=0.5, scale=0.2,size=size)
        self._strengths = list(strengths) # So we can simply pop()
        
    def get_strength(self):
        """Pops a strength from our saved sampling distribution"""
        return self._strengths.pop()
    
    def __len__(self): # Just so we can create an iterable. Probs not useful
        return len(self._strengths)
    
    def __getitem__(self, val):  # Just so we can create an iterable. Probs not useful
        return self._strengths[val]

class Animal:
    def __init__(self, position, strength, born):
        self._is_female = choose_gender()
        self._strength = strength
        self._position = position
        self._born = born
        self._processed = False
        
    def get_processed(self):
        return self._processed
    
    def get_is_born(self):
        return self._born
    
    def set_processed(self):
        self._processed = True
    
    def reset_processed(self):
        self._processed = False

    def get_position(self):
        return self._position
    
    def _set_position(self, val):
        """Brutish method to set the position of a creature. Not for general consumption!"""
        self._position = val
    
    def get_is_female(self):
        return self._is_female
    
    def get_strength(self):
        return self._strength

    def have_a_baby(self, other):
        """Creates a new instance of specified type in closest empty position to self.
        Relies on function find_free()
        Relies on Animal class member methods:
        get_position(), get_strength(), set_processed()
        """
          
        empty_pos = find_free(river, self.get_position())
        
        if empty_pos == -1:
            global halt_simulation
            halt_simulation = "River is full!"
            return
        
        new_strength = np.mean([self.get_strength(), other.get_strength()])
        
        if isinstance(self, Bear):
            if debug: logger.debug(f"Creating Bear with born=True at {empty_pos}")
            river[empty_pos] = Bear(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()
            
        elif isinstance(self, Fish):
            if debug: logger.debug(f"Creating Fish with born=True, strength={new_strength}, at {empty_pos}")
            river[empty_pos] = Fish(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()

        
    def __add__(self, other):
        """Operator override to allow straightforward addition of objects.
        Doesn't accept None as an argument. Reliant on Animal class member methods:
        get_position(), get_is_female(), have_a_baby(), set_processed()"""
        
        self_position, other_position = self.get_position(), other.get_position()
        
        if type(self) == type(other): # Animals are the same type
            
            if not self.get_is_female() and other.get_is_female(): # not has higher priority than boolean operation and
            
                if debug: logger.debug(f"T{t}: Calling have_a_baby between {type(self), self_position} and {type(other), other_position}")
                self.have_a_baby(other)
                self.set_processed()
                other.set_processed()
                return "A new beast is born" # The creature is the same type
            else:
                # Different gender beasts are trying to enter the same position
                # self_strength, other_strength = self.get_strength(), other.get_strength()
                # if self_strength >= 
                
                pass

        elif isinstance(self, Bear): # bear eats fish
            #river.insert(other_position, river.pop(self_position))
            
            if debug: logger.debug(f"T{t}: Bear at {self_position} eats fish at {other_position}")
            
            river[other_position] = river[self_position]
            
            river[self_position] = None
            river[other_position]._set_position(other_position)
            river[other_position].set_processed()
                        
            return "There's been a murder" 

        elif isinstance(self, Fish): # fish moves in with bear, so disappears
            
            if debug: logger.debug(f"T{t}: Fish at {self_position} moves in with Bear, so is eaten")
            
            river[self_position] = None
            # self.set_processed() # Unecessary as we've removed ref to self from river
            
            return "Kamikaze"
            
    def move(self, direction):
        """Reliant on Animal class member methods:
        _set_position(), get_position(), .set_processed()"""
        
        self_position = self.get_position()
        
        if debug: logger.debug(f"T{t}: Move from {self_position}, {direction}")
        
        if direction == 'upstream' and self_position == 0:
            return # Already at source. Can't move upstream
    
        elif direction == 'downstream' and self_position == len(river)-1:
            return # Already at mouth. Can't move downstream

        if direction == 'upstream':
            target_position = self_position - 1
        
        elif direction == 'downstream':
            target_position = self_position + 1

        if river[target_position]: # There's a creature in target location
            
            if debug: logger.debug(f"T{t}: Add {self_position} to {target_position}")
            
            river[self_position] + river[target_position]
        
        else: # There's a None in target location 

            # river.insert(target_position, river.pop(self_position))
            
            river[target_position] = river[self_position]
            river[target_position]._set_position(target_position)
            river[target_position].set_processed()
            river[self_position] = None
            
            
            if debug: logger.debug(f"T{t}: Move to {target_position}")
    
    def time_step(self):
        """Reliant on Animal calss member methods:
        move()"""
        
        #print("time_step(self)",self)
        moves = bool(round(random.random())) # Does the creature move?
        
        if moves:
            up = bool(round(random.random())) # Does the creature move up or downstream?
            self.move('upstream') if up else self.move('downstream')

    def __repr__(self):
        
        # elif isinstance(item, Fish):
        #     
        # elif isinstance(item, Bear):
        #     print('\x1b[31mb\x1b[0m', end='') if item.get_is_born() and debug else print('\x1b[31mB\x1b[0m', end='')
        
        gender_symbol = GENDER_SYMBOL_FEMALE if self.get_is_female() else GENDER_SYMBOL_MALE
        gender_symbol = GENDER_SYMBOL_FEMALE if self.get_is_female() else GENDER_SYMBOL_MALE
        
        if isinstance(self, Bear):
            if self.get_is_born() and debug:
                return 'b' + gender_symbol
            else:
                return 'B' + gender_symbol
            
        elif isinstance(self, Fish):
            if self.get_is_born() and debug:
                return 'f' + gender_symbol
            else:
                return 'F' + gender_symbol
            
    def __len__(self):
        return len(self.__repr__())
            
class Bear(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)
        

class Fish(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)

def choose_gender():
    return bool(np.random.binomial(n=1, p=P_FEMALE, size=int(1)))


# FUNCTIONS

# def show_strength(text,strength_val):
#     colour_key = int(round(strength_val, 1)*10)
#     return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def show_strength(text, strength_val, transform = False):
    # print('colour_key', colour_key)
    if transform:
        s = gaussian(GAUSSIAN_A, GAUSSIAN_B, GAUSSIAN_C, strength_val)
        s = 0.1 if s < 0.1 else s
    colour_key = int((round(s, 1))*NUM_SHADES) - 1
    return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def display_river(t):
    print(str(t).zfill(3), end=' ')
    for i, item in enumerate(river):
        if not item:
            print('--', end='')
        else:
            s = item.get_strength()
            print(show_strength(item, s, transform = True), end='')
        print(' ', end='')
    print('')

def find_free(l, pos):
    '''Returns index value of closest None value to a specified position in a list.
    Returns -1 if no None values are found'''
    
    offset, left_done, right_done, length = 0, False, False, len(l)
    
    while not (right_done and left_done):
        
        offset += 1
        
        pos_to_right = pos + offset
        pos_to_left = pos - offset
        
        if pos_to_right <= length-1:
            if not l[pos_to_right]:
                return pos_to_right
        else:
            right_done = True
        
        if pos_to_left >= 0:
            if not l[pos_to_left]:
                return pos_to_left
        else:
            left_done = True

    return -1

def gaussian(a, b, c, x):
    """Gaussian transformation function
    a: height of the curve's peak
    b: centre
    c: standard deviation
    x: integer"""
    
    power = (-(x-b)**2)/(2*c**2)
    return a * np.exp(power)

# MAIN PROGRAM

# CONSTANTS

RIVER_LENGTH = 50
DEFAULT_IS_FEMALE = False
P_BEAR = 0.2
P_FISH = 0.2
P_FEMALE = 0.7
GENDER_SYMBOL_FEMALE = '\u2640'
GENDER_SYMBOL_MALE = '\u2642'

# colouring/shading
GAUSSIAN_A = 1   # height of the curve's peak
GAUSSIAN_B = 1 # centre
GAUSSIAN_C = 0.50 # standard deviation

COLOUR_BRACKET_MIN = 235 # 196 Red
COLOUR_BRACKET_MAX = 255 # 201 Pink
NUM_SHADES = 10

# Initialise colouring of StdOut

step_size = int((COLOUR_BRACKET_MAX-COLOUR_BRACKET_MIN)/NUM_SHADES)
colour_range = range(COLOUR_BRACKET_MAX, COLOUR_BRACKET_MIN, -step_size)

# Create empty river
river = [None] * RIVER_LENGTH
rivers = list()
halt_simulation = False

log = []

# Populate river
strengths = Strengths(RIVER_LENGTH)

for i in range(len(river)):
    r = random.choices(['bear', 'fish', None], [P_BEAR, P_FISH, 1-(P_BEAR + P_FISH)])[0]
    if r == 'bear':
        river[i] = Bear(i, strength = strengths.get_strength())
    elif r == 'fish':
        river[i] = Fish(i, strength = strengths.get_strength())
    else:
        river[i] = None

# Here we run our simulation

debug = True

t = 0

def display_positions_labels():
    length_floor_10 = RIVER_LENGTH // 10
    labels = "".join([str(x)+'  ' for x in range(length_floor_10)])
    print("Tx ", (9 * " ").join(labels))

    
# print example of shading
for i in range(10):
    i = i/10
    print(show_strength('B', i, transform = True), end=' ')
print()

display_positions_labels()
display_river(t)

while not halt_simulation:
    t += 1
    if t % 10 == 0 and debug: display_positions_labels()
    display_river(t) # t specifies the first character of the row
    for position in river: # iterate through all elements in river
        if position: # ... that aren't None
            #print(position)
            if not position.get_processed(): # ... that haven't been processed already
                    position.time_step()
    
    for position in river:
        if position:
            position.reset_processed()
    
    log.append(["----- COMPLETED TIME STEP -----"])      
    
display_river(t + 1)
print('Simulation halted because:', halt_simulation)

In [2]:
######################################################
######################################################
###
### And now implement strengths
###
######################################################
######################################################

# IMPORTS

import random
import functools
import numpy as np

# LOGGER

import sys
from loguru import logger

logger.remove() # clear the decks
logger.add("debug.log", mode="w", backtrace=False) # specify mode="w" to overwrite each time, or a to append

logger.info("NEW RUN STARTED")

# CLASSES

class Strengths:
    """Generates a set of strengths for creatures in the river system
    Allows each strength value to be retrieved and hence assigned to each creature during
    initialisation of the system"""
    
    def __init__(self, size):
        
        """Generates a sampling distribution based on the Normal distribution"""
        count = 0
        strengths = [-1,2]
        while min(strengths) <= 0 or max(strengths) >= 1: # Ensures we only generate a distribution with values within acceptable range
            count += 1
            if count > 10: raise Exception("Too many attempts to build a normal distribution")
            strengths = np.random.normal(loc=0.5, scale=0.2,size=size)
        self._strengths = list(strengths) # So we can simply pop()
        
    def get_strength(self):
        """Pops a strength from our saved sampling distribution"""
        return self._strengths.pop()
    
    def __len__(self): # Just so we can create an iterable. Probs not useful
        return len(self._strengths)
    
    def __getitem__(self, val):  # Just so we can create an iterable. Probs not useful
        return self._strengths[val]

class Animal:
    def __init__(self, position, strength, born):
        self._is_female = choose_gender()
        self._strength = strength
        self._position = position
        self._born = born
        self._processed = False
        
    def get_processed(self):
        return self._processed
    
    def get_is_born(self):
        return self._born
    
    def set_processed(self):
        self._processed = True
    
    def reset_processed(self):
        self._processed = False

    def get_position(self):
        return self._position
    
    def _set_position(self, val):
        """Brutish method to set the position of a creature. Not for general consumption!"""
        self._position = val
        self.set_processed()
    
    def get_is_female(self):
        return self._is_female
    
    def get_strength(self):
        return self._strength

    def have_a_baby(self, other):
        """Creates a new instance of specified type in closest empty position to self.
        Strength of new instance to be average of its parents.
        Relies on function find_free()
        Relies on Animal class member methods:
        get_position(), get_strength(), set_processed()
        """
          
        empty_pos = find_free(river, self.get_position())
        
        if empty_pos == -1:
            global halt_simulation
            halt_simulation = "River is full!"
            return
        
        new_strength = np.mean([self.get_strength(), other.get_strength()])
        
        if isinstance(self, Bear):
            if debug: logger.debug(f"Creating Bear with born=True at {empty_pos}")
            river[empty_pos] = Bear(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()
            
        elif isinstance(self, Fish):
            if debug: logger.debug(f"Creating Fish with born=True, strength={new_strength}, at {empty_pos}")
            river[empty_pos] = Fish(empty_pos, strength = new_strength, born=True)
            river[empty_pos].set_processed()

        
    def __add__(self, other):
        """Operator override to allow straightforward addition of objects.
        Doesn't accept None as an argument. Reliant on Animal class member methods:
        get_position(), get_is_female(), have_a_baby(), set_processed()"""
        
        self_position, other_position = self.get_position(), other.get_position()
        
        if type(self) == type(other): # Animals are the same type
            
            if not self.get_is_female() and other.get_is_female(): # not has higher priority than boolean operation and
            
                if debug: logger.debug(f"T{t}: Calling have_a_baby between {type(self), self_position} and {type(other), other_position}")
                self.have_a_baby(other)
                self.set_processed()
                other.set_processed()
                return "A new beast is born" # The creature is the same type
            else:
                # Beasts of the same type and gender are trying to move in to the same spot.
                # If self is stronger, it moves to other's position and its original pos is set to None
                # If other is stronger, the reverse happens.
                # If they are of equal strength thenthere's no change.
                
                self_strength, other_strength = self.get_strength(), other.get_strength()
                
                if self_strength > other_strength:
                    
                    river[other_position] = river[self_position]; river[self_position] = None
                    river[other_position]._set_position(other_position); river[other_position].set_processed()
                    
                elif other_strength > self_strength:
                    
                    river[self_position] = river[other_position]; river[other_position] = None
                    river[self_position]._set_position(self_position); river[self_position].set_processed()
                
                else:
                    pass

        elif isinstance(self, Bear): # bear eats fish
            #river.insert(other_position, river.pop(self_position))
            
            if debug: logger.debug(f"T{t}: Bear at {self_position} eats fish at {other_position}")
            
            river[other_position] = river[self_position]; river[self_position] = None
            river[other_position]._set_position(other_position); river[other_position].set_processed()
                        
            return "There's been a murder" 

        elif isinstance(self, Fish): # fish moves in with bear, so disappears
            
            if debug: logger.debug(f"T{t}: Fish at {self_position} moves in with Bear, so is eaten")
            
            river[self_position] = None
            # self.set_processed() # Unecessary as we've removed ref to self from river
            
            return "Kamikaze"
            
    def move(self, direction):
        """Reliant on Animal class member methods:
        _set_position(), get_position(), .set_processed()"""
        
        self_position = self.get_position()
        
        if debug: logger.debug(f"T{t}: Move from {self_position}, {direction}")
        
        if direction == 'upstream' and self_position == 0:
            return # Already at source. Can't move upstream
    
        elif direction == 'downstream' and self_position == len(river)-1:
            return # Already at mouth. Can't move downstream

        if direction == 'upstream':
            target_position = self_position - 1
        
        elif direction == 'downstream':
            target_position = self_position + 1

        if river[target_position]: # There's a creature in target location
            
            if debug: logger.debug(f"T{t}: Add {self_position} to {target_position}")
            
            river[self_position] + river[target_position]
        
        else: # There's a None in target location 

            #river[target_position] = river[self_position]
            #river[self_position] = None            
            
            # Use simultaneous assignment to achieve the same effect as above couple of lines
            river[target_position], river[self_position] = river[self_position], river[target_position]
            
            river[target_position]._set_position(target_position)

            if debug: logger.debug(f"T{t}: Move to {target_position}")
    
    def time_step(self):
        """Reliant on Animal class member methods:
        move()"""
        
        #print("time_step(self)",self)
        moves = bool(round(random.random())) # Does the creature move?
        
        if moves:
            up = bool(round(random.random())) # Does the creature move up or downstream?
            self.move('upstream') if up else self.move('downstream')

    def __repr__(self):
        
        # elif isinstance(item, Fish):
        #     
        # elif isinstance(item, Bear):
        #     print('\x1b[31mb\x1b[0m', end='') if item.get_is_born() and debug else print('\x1b[31mB\x1b[0m', end='')
        
        gender_symbol = GENDER_SYMBOL_FEMALE if self.get_is_female() else GENDER_SYMBOL_MALE
        
        if isinstance(self, Bear):
            if self.get_is_born() and debug:
                return 'b' + gender_symbol
            else:
                return 'B' + gender_symbol
            
        elif isinstance(self, Fish):
            if self.get_is_born() and debug:
                return 'f' + gender_symbol
            else:
                return 'F' + gender_symbol
            
    def __len__(self):
        return len(self.__repr__())
            
class Bear(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)
        

class Fish(Animal):
    def __init__(self, position, strength, born = False):
        super().__init__(position, strength, born)

def choose_gender():
    return bool(np.random.binomial(n=1, p=P_FEMALE, size=int(1)))


# FUNCTIONS

# def show_strength(text,strength_val):
#     colour_key = int(round(strength_val, 1)*10)
#     return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def show_strength(text, strength_val, transform = False):
    # print('colour_key', colour_key)
    if transform:
        s = gaussian(GAUSSIAN_A, GAUSSIAN_B, GAUSSIAN_C, strength_val)
        s = 0.1 if s < 0.1 else s
    colour_key = int((round(s, 1))*NUM_SHADES) - 1
    return "\x1b[38;5;{}m{}\x1b[0m".format(colour_range[colour_key], text)

def display_river(t):
    print(str(t).zfill(3), end=' ')
    for i, item in enumerate(river):
        if not item:
            print('--', end='')
        else:
            s = item.get_strength()
            print(show_strength(item, s, transform = True), end='')
        print(' ', end='')
    print('')

def find_free(l, pos):
    '''Returns index value of closest None value to a specified position in a list.
    Returns -1 if no None values are found'''
    
    offset, left_done, right_done, length = 0, False, False, len(l)
    
    while not (right_done and left_done):
        
        offset += 1
        
        pos_to_right = pos + offset
        pos_to_left = pos - offset
        
        if pos_to_right <= length-1:
            if not l[pos_to_right]:
                return pos_to_right
        else:
            right_done = True
        
        if pos_to_left >= 0:
            if not l[pos_to_left]:
                return pos_to_left
        else:
            left_done = True

    return -1

def gaussian(a, b, c, x):
    """Gaussian transformation function
    a: height of the curve's peak
    b: centre
    c: standard deviation
    x: integer"""
    
    power = (-(x-b)**2)/(2*c**2)
    return a * np.exp(power)

# MAIN PROGRAM

# CONSTANTS

RIVER_LENGTH = 20
DEFAULT_IS_FEMALE = False
P_BEAR = 0.4
P_FISH = 0.4
P_FEMALE = 0.7
GENDER_SYMBOL_FEMALE = '\u2640'
GENDER_SYMBOL_MALE = '\u2642'

# colouring/shading
GAUSSIAN_A = 1   # height of the curve's peak
GAUSSIAN_B = 1 # centre
GAUSSIAN_C = 0.50 # standard deviation

COLOUR_BRACKET_MIN = 235 # 196 Red
COLOUR_BRACKET_MAX = 255 # 201 Pink
NUM_SHADES = 10

# Initialise colouring of StdOut

step_size = int((COLOUR_BRACKET_MAX-COLOUR_BRACKET_MIN)/NUM_SHADES)
colour_range = range(COLOUR_BRACKET_MAX, COLOUR_BRACKET_MIN, -step_size)

# Create empty river
river = [None] * RIVER_LENGTH
rivers = list()
halt_simulation = False

log = []

# Populate river
strengths = Strengths(RIVER_LENGTH)

for i in range(len(river)):
    r = random.choices(['bear', 'fish', None], [P_BEAR, P_FISH, 1-(P_BEAR + P_FISH)])[0]
    if r == 'bear':
        river[i] = Bear(i, strength = strengths.get_strength())
    elif r == 'fish':
        river[i] = Fish(i, strength = strengths.get_strength())
    else:
        river[i] = None

# Here we run our simulation

debug = True

t = 0

def display_positions_labels():
    length_floor_10 = RIVER_LENGTH // 10
    labels = "".join([str(x)+'  ' for x in range(length_floor_10)])
    print("Tx ", (9 * " ").join(labels))

    
# print example of shading
for i in range(10):
    i = i/10
    print(show_strength('B', i, transform = True), end=' ')
print()

display_positions_labels()
display_river(t)

while not halt_simulation:
    t += 1
    if t % 10 == 0 and debug: display_positions_labels()
    if t == 100: halt_simulation = 't = 100'
    display_river(t) # t specifies the first character of the row
    for position in river: # iterate through all elements in river
        if position: # ... that aren't None
            #print(position)
            if not position.get_processed(): # ... that haven't been processed already
                    position.time_step()
    
    for position in river:
        if position:
            position.reset_processed()
    
    if debug: log.append(["----- COMPLETED TIME STEP -----"])      
    
display_river(t + 1)
print('Simulation halted because:', halt_simulation)

B B B B B B B B B B 
Tx  0                             1                    
000 F♀ -- B♂ B♀ B♀ B♂ B♀ F♀ B♂ -- B♀ -- B♀ -- F♀ -- -- -- F♀ B♂ 
001 F♀ -- B♂ B♀ B♀ B♂ B♀ F♀ B♂ -- B♀ -- B♀ -- F♀ -- -- -- F♀ B♂ 
002 F♀ -- -- B♂ -- B♀ B♀ -- B♂ -- B♀ B♀ -- -- F♀ -- -- -- F♀ B♂ 
003 F♀ -- -- B♂ -- B♀ -- B♀ B♂ -- b♀ -- B♀ F♀ -- -- -- -- F♀ B♂ 
004 F♀ -- B♂ -- -- -- B♀ B♀ B♂ -- b♀ -- -- B♀ -- -- -- -- B♂ -- 
005 F♀ -- -- B♂ -- -- B♀ -- B♀ -- b♀ -- -- -- B♀ -- -- B♂ -- -- 
006 F♀ -- B♂ -- -- -- B♀ B♀ -- -- -- b♀ -- -- -- B♀ B♂ -- -- -- 
007 F♀ -- B♂ -- -- -- B♀ -- -- -- -- b♀ -- -- -- B♀ -- B♂ -- -- 
008 -- F♀ -- B♂ -- -- -- B♀ -- -- b♀ -- -- -- -- B♀ B♂ -- -- -- 
009 F♀ -- -- B♂ -- -- -- B♀ -- b♀ -- -- -- -- -- B♀ -- B♂ -- -- 
Tx  0                             1                    
010 -- F♀ -- -- B♂ -- -- B♀ -- -- b♀ -- -- -- -- -- B♀ B♂ -- -- 
011 F♀ -- -- B♂ -- -- B♀ -- -- -- b♀ -- -- -- -- -- B♀ B♂ b♀ -- 
012 F♀ -- -- B♂ -- B♀ -- -- -- -- -- b♀ -- -- -- -- -- B♀ b♀ -- 
013 -- F♀ -- B♂ -- -- 

In [ ]:
# unique_ids = []

# def get_unique_id():
    # """Issues and records unique IDs.
    # Doesn't work very well because the comparison r not in doesn't take into account the zfill operation"""
#     r = ''
#     while r not in unique_ids:
#         r = str(round(random.random()*1000)).zfill(3)
#         unique_ids.append(r)
#     return r

In [ ]:
#     def move_upstream(self):
        
#         position = self.get_position()
        
#         if position == 0: # Already at source. Can't move upstream
#             return
        
#         else:
#             # move upstream
            
#             if river[position-1]: # There's a creature in target location
#                 # Alter data type to include operator override and then we can simply add positions here and the member functions will take care of logic
#                 if river[position-1].get_type() == self.get_type(): # The creature is the same type
#                     self.have_a_baby()
                    
#                 elif (river[position-1].get_type() == 'bear') and (self.get_type() == 'fish'):
#                     self.die()

#                 elif (river[position-1].get_type() == 'fish') and (self.get_type() == 'bear'):  # Defining __add__ & __radd__ will remove this step
#                     river[position-1].die()

#             else:
#                 self._position -= 1
#                 river[position-1] = river[position]
#                 river[position] = None
        

#     def move_downstream(self):
        
#         position = self.get_position()
        
#         if position == len(river)-1: # Already at mouth. Can't move downstream
#             pass
        
#         else:
#             # move downstream
#             if river[position+1]:
#                 if river[position+1].get_type() == self.get_type():
#                     self.have_a_baby()
                    
#                 elif (river[position+1].get_type() == 'bear') and (self.get_type() == 'fish'):
#                     self.die()

#                 elif (river[position+1].get_type() == 'fish') and (self.get_type() == 'bear'):
#                     river[position+1].die()

#             else:
#                 self._position += 1
#                 river[position+1] = river[position]
#                 river[position] = None

In [ ]:
# Before merging the below:

#         if direction == 'upstream':

#             if self_position == 0: return # Already at source. Can't move upstream

#             target = river[self_position-1]
            
#             if target: self + target # There's a creature in target location

#             else: self.set_position(self_position - 1) # There's a None in target location 

#         elif direction == 'downstream':

#             if self_position == len(river)-1: return # Already at mouth. Can't move downstream
        
#             target = river[self_position+1]

#             if target: self + target

#             else: self.set_position(self_position + 1)